# FINDATA 01 - 종목 마스터

(stock_master 테이블)

#### 이승준 http://fb.com/plusjune

# 상장회사검색

* 거래소 상장회사검색 http://marketdata.krx.co.kr/contents/MKD/04/0406/04060100/MKD04060100.jsp
* 거래소 csv 읽기 에러: csv로 다운로드 하는 경우 주소 등의 컬럼에 '\n' 데이터가 포함
* 엑셀로 다운로드: xlrd (엑셀 입출력 패키지)를 설치

```
$ sudo pip install xlrd
```

* KRX 01 종목코드 전체 https://gist.github.com/48486c12915d197714bc

In [206]:
import requests
import pandas as pd
import io

def get_krx_stock_master():
    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0406/04060100/mkd04060100_01',
        'market_gubun':'ALL', # ''ALL':전체, STK': 코스피
        'isu_cdnm':'전체',
        'isu_cd':'',
        'isu_nm':'',
        'isu_srt_cd':'',
        'sort_type':'A',
        'std_ind_cd':'01',
        'par_pr':'',
        'cpta_scl':'',
        'sttl_trm':'',
        'lst_stk_vl':'1',
        'in_lst_stk_vl':'',
        'in_lst_stk_vl2':'',
        'cpt':'1',
        'in_cpt':'',
        'in_cpt2':'',
        'pagePath':'/contents/MKD/04/0406/04060100/MKD04060100.jsp',
    }

    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content

    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }

    r = requests.post(down_url, down_data)
    f = io.BytesIO(r.content)
    
    usecols = ['종목코드', '기업명', '업종코드', '업종', '대표전화', '주소']
    df = pd.read_excel(f, converters={'종목코드': str, '업종코드': str}, usecols=usecols)
    df.columns = ['code', 'name', 'sector_code', 'sector', 'telephone', 'address']
    return df

In [207]:
df = get_krx_stock_master()
df.head()

,code,name,sector_code,sector,telephone,address
0,060310,3S,032602,전자부품 제조업,02-896-9474,서울특별시 금천구 시흥대로71길 30-1
1,095570,AJ네트웍스,126903,산업용 기계 및 장비 임대업,02-6240-0800,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
2,068400,AJ렌터카,126901,운송장비 임대업,1544-1600,서울특별시 구로구 서부샛길 822
3,006840,AK홀딩스,137105,"회사본부, 지주회사 및 경영컨설팅 서비스업",02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동)
4,054620,AP시스템,032902,특수 목적용 기계 제조업,031-379-2700,경기도 화성시 동탄면 동탄산단8길 15-5


In [213]:
import sqlite3
conn = sqlite3.connect('findata.db')

In [214]:
# stock_master 테이블  쓰기

df_master = df[['code', 'name', 'sector_code', 'sector']]
df_master.to_sql('stock_master', conn, if_exists='replace', index=False)

In [215]:
# stock_desc 테이블  쓰기
df_desc = df[['code', 'name', 'telephone', 'address']].copy()
df_desc['market'] = ''
df_desc['wics'] = ''
df_desc['name_en'] = ''
df_desc['desc'] = ''
df_desc['desc_date'] = ''

df_desc.to_sql('stock_desc', conn, if_exists='replace', index=False)

# 데이터 확인

In [216]:
# 테이블  읽기
df = pd.read_sql_query('SELECT * FROM stock_master limit 5', conn)
df.head()

,code,name,sector_code,sector
0,060310,3S,032602,전자부품 제조업
1,095570,AJ네트웍스,126903,산업용 기계 및 장비 임대업
2,068400,AJ렌터카,126901,운송장비 임대업
3,006840,AK홀딩스,137105,"회사본부, 지주회사 및 경영컨설팅 서비스업"
4,054620,AP시스템,032902,특수 목적용 기계 제조업


In [217]:
# 테이블  읽기
df = pd.read_sql_query('SELECT * FROM stock_desc limit 5', conn)
df.head()

,code,name,telephone,address,market,wics,name_en,desc,desc_date
0,060310,3S,02-896-9474,서울특별시 금천구 시흥대로71길 30-1,,,,,
1,095570,AJ네트웍스,02-6240-0800,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",,,,,
2,068400,AJ렌터카,1544-1600,서울특별시 구로구 서부샛길 822,,,,,
3,006840,AK홀딩스,02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동),,,,,
4,054620,AP시스템,031-379-2700,경기도 화성시 동탄면 동탄산단8길 15-5,,,,,


In [218]:
df = pd.read_sql_query(
    'SELECT m.code, m.name, d.address '
    'FROM stock_master AS m ' 
    'INNER JOIN  stock_desc AS d '
    'ON m.code = d.code', 
    conn)

df.head()

,code,name,address
0,060310,3S,서울특별시 금천구 시흥대로71길 30-1
1,095570,AJ네트웍스,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
2,068400,AJ렌터카,서울특별시 구로구 서부샛길 822
3,006840,AK홀딩스,서울특별시 구로구 구로중앙로 152(구로동)
4,054620,AP시스템,경기도 화성시 동탄면 동탄산단8길 15-5


In [219]:
conn.close()